In [1]:
# how to open xlsx file

import pandas as pd
import plotly.express as px
import numpy as np

In [35]:
df = pd.read_excel(
    "https://censusindia.gov.in/nada/index.php/catalog/10191/download/13303/DDW-C16-STMT-MDDS-0000.XLSX",
    skiprows=[1, 2, 3, 4, 5],
)
df.columns = [
    "table_name",
    "state_code",
    "district_code",
    "sub_district_code",
    "area_name",
    "code",
    "name",
    "total",
    "total_male",
    "total_female",
    "rural",
    "rural_male",
    "rural_female",
    "urban",
    "urban_male",
    "urban_female",
]
df = df.loc[df["area_name"] == "INDIA"].drop(
    columns=[
        "table_name",
        "state_code",
        "district_code",
        "sub_district_code",
        "area_name",
    ]
)

In [36]:
languages = df.loc[df.code % 1000 == 0, ["code", "name"]]
# extract the non-numeral part of the name
languages["name"] = languages["name"].str.extract(r"([A-Za-z\s]+)")
languages["name"] = languages["name"].str.strip().str.capitalize()
languages = languages.set_index("code")

In [4]:
# drop the total row
df = df.loc[df.code % 1000 != 0, :]
# Add a column for the language name
df["language"] = df["code"].apply(lambda w: languages.loc[w // 1000 * 1000, "name"])
df["name"] = df["name"].str.strip().str.extract(r"([A-Za-z\s]+)")
df["name"] = df["name"].str.strip()
# move the language column to the front
df = df[
    [
        "code",
        "language",
        "name",
        "total",
        "total_male",
        "total_female",
        "rural",
        "rural_male",
        "rural_female",
        "urban",
        "urban_male",
        "urban_female",
    ]
].rename(columns={"name": "sub_language"})

In [5]:
df

,code,language,sub_language,total,total_male,total_female,rural,rural_male,rural_female,urban,urban_male,urban_female
1,1002,Assamese,Assamese,14816414,7555350,7261064,12680101,6458345,6221756,2136313,1097005,1039308
2,1999,Assamese,Others,494937,255233,239704,464600,239370,225230,30337,15863,14474
4,2007,Bengali,Bengali,96177835,49254753,46923082,68921285,35356084,33565201,27256550,13898669,13357881
5,2011,Bengali,Chakma,228281,116928,111353,220162,112745,107417,8119,4183,3936
6,2015,Bengali,Haijong,71792,36364,35428,65601,33243,32358,6191,3121,3070
...,...,...,...,...,...,...,...,...,...,...,...,...
468,120999,Yimchungre,Others,3350,1746,1604,3333,1737,1596,17,9,8
470,121001,Zeliang,Zeliang,63529,32144,31385,49744,25524,24220,13785,6620,7165
472,122003,Zemi,Zemi,50923,25869,25054,45036,22889,22147,5887,2980,2907
473,122999,Zemi,Others,2,0,2,1,0,1,1,0,1


In [34]:
languages

,name
code,
1000,Assamese
2000,Bengali
3000,Bodo
4000,Dogri
5000,Gujarati
...,...
120000,Yimchungre
121000,Zeliang
122000,Zemi


In [6]:
df.to_csv("./data/mother_tongues.csv", index=False)

In [7]:
df["pct"] = df["total"] / df["total"].sum()

In [16]:
fig = px.treemap(
    df,
    path=[px.Constant("India"), "language", "sub_language"],
    values="total",
)
fig.update_layout(
    margin=dict(t=50, l=25, r=25, b=25),
)
fig.show()

In [9]:
df.loc[df["language"] == "HINDI"].total.sum() / df.total.sum()

0.0

In [31]:
data = {
    "name": "India",
    "children": [
        {
            "name": lang,
            "children": grp[["sub_language", "total"]].to_dict(orient="records"),
        }
        for lang, grp in df.groupby("language")
    ],
}

In [33]:
import json

with open("./data/tree_map.json", "w") as f:
    json.dump(data, f)